In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import pickle

NBA_learn_df=pd.read_csv("../Resources/NBA_learning2.csv")
NBA_learn_df

Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'compat'

In [ ]:
NBA_learn_df.sample(frac=1).reset_index(drop=True)

In [ ]:
X=NBA_learn_df.drop(columns=["Unnamed: 0","Players ok"])
y=NBA_learn_df['Players ok']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
import keras
print(keras.__version__)

In [ ]:
mod1 = Sequential() 
mod1.add(Dense(50, activation='relu',input_shape=(16,)))
mod1.add(Dropout(0.3))
mod1.add(Dense(100, activation='relu'))
#mod1.add(Dropout(0.3))
mod1.add(Dense(50, activation='relu'))

mod1.add(Dense(1, activation="sigmoid"))

mod1.summary()

In [ ]:
from keras.optimizers import SGD
from keras import metrics
from keras.metrics import Recall
opt = SGD(lr=0.0001)
#model.compile(loss = "categorical_crossentropy", optimizer = opt)


mod1.compile(loss='binary_crossentropy', optimizer = opt, metrics=['binary_accuracy',Recall(thresholds=[0.3])])#thresholds=[0.5,0.5,0.5],


checkpointer = ModelCheckpoint(filepath='NBA.h5', monitor='loss',verbose=0, save_best_only='min', period=1)

h = mod1.fit(X_train_scaled, encoded_y_train, batch_size=1000, epochs=100, verbose=1, validation_data = (X_test_scaled,encoded_y_test),callbacks=[checkpointer])

In [ ]:
model_loss, model_accuracy,recall = mod1.evaluate(
    X_test_scaled, encoded_y_test, verbose=0)
print("Normal Neural Network - Loss:" +str(model_loss)+", Accuracy: "+str(model_accuracy)+ " Recall: "+str(recall))
#mod1.evaluate(    X_test_scaled, encoded_y_test, verbose=0)

In [ ]:
encoded_predictions = mod1.predict_classes(X_test_scaled[:1000])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print("Predicted classes:"+ str(prediction_labels))
print("Actual Labels: "+str(list(y_test[:1000])))

In [ ]:
# save the model to disk
filename = 'finalized_model.pkl'
pickle.dump(mod1, open(filename, 'wb'))

In [ ]:
NBA_learn_df=NBA_learn_df.dropna()
X=NBA_learn_df.drop(columns=["Unnamed: 0","PTS"])
y=NBA_learn_df['PTS']


In [ ]:


X_scaler = MinMaxScaler().fit(X)
X_scaled = X_scaler.transform(X)




y=pd.DataFrame(y)
y_scaler = MinMaxScaler().fit(y)#.reshape(-1, 1)
y_scaled = y_scaler.transform(y)




X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, random_state=1)

In [ ]:
mod2 = Sequential() 
mod2.add(Dense(50, activation='relu',input_shape=(16,)))
#mod2.add(Dropout(0.3))
mod2.add(Dense(100, activation='relu'))
#mod2.add(Dropout(0.3))
mod2.add(Dense(50, activation='relu'))

mod2.add(Dense(1, activation="relu"))

mod2.summary()

In [ ]:
from keras import metrics
#from keras.metrics import MeanRelativeError

opt = SGD(lr=0.1)
mod2.compile(loss="mse", optimizer = opt,metrics=["mean_absolute_percentage_error"])#, metrics=['accuracy']

checkpointer = ModelCheckpoint(filepath='NBA2.h5', monitor='loss',verbose=0, save_best_only='min', period=1)

h = mod2.fit(X_train, y_train, batch_size=1000, epochs=1000, verbose=1, validation_data = (X_test,y_test),callbacks=[checkpointer])

In [ ]:
model_loss, model_accuracy = mod2.evaluate(
    X_test, y_test, verbose=0)
print("Normal Neural Network - val_loss:" +str(model_loss)+", val_mean_absolute_percentage_error: "+str(model_accuracy))

In [ ]:
 mod2.evaluate(X_test, y_test, verbose=0)

In [ ]:
# save the model 2 to disk
filename2 = 'finalized_model_2.pkl'
pickle.dump(mod2, open(filename2, 'wb'))